In [13]:
import numpy as np

x = np.array(([3,5],[5,1],[10,2]),dtype=float)
y = np.array(([75],[82],[93]),dtype=float)
x = x/np.amax(x,axis=0)
y = y/100

class Neural_Network(object):
    def __init__(self):
        self.inputLayerSize = 2
        self.outputLayerSize = 1
        self.hiddenLayerSize = 3
        
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self,x):
        self.z2 = np.dot(x,self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2,self.W2)
        yHat = self.sigmoid(self.z3)
        return yHat
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
        

In [14]:
NN = Neural_Network()

In [15]:
ytest = NN.forward(x)

In [16]:
ytest


array([[ 0.38890965],
       [ 0.4062436 ],
       [ 0.37130494]])

In [17]:
y

array([[ 0.75],
       [ 0.82],
       [ 0.93]])